In [ ]:
!pip install optuna
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from dataclasses import dataclass
from enum import Enum
import math
import sys
import os
import optuna
import time
import joblib
import multiprocessing
import statistics
from subprocess import run, PIPE

# visualizerからテストをとってきた個数
n_files = 100
root = "/content/drive/MyDrive/"
#コンテスト毎に分けておく
contest_name = "ahc_029/"
#得点出力するコード
program = "calc_score.py"

def calc_score_each(seed: int, wait_turn: int,can_per_boarder: float):

    #それぞれのテスト毎に読み込み実行させる
    in_file = f"{root + contest_name}in/{seed:04}.txt"
    changed_in_file = f"{root + contest_name}in/ch{seed:04}.txt"
    #out_file = f"{root + contest_name}out/{seed:04}.txt"
    os.system(f'echo "{wait_turn} {can_per_boarder}" > {changed_in_file}')
    os.system(f'cat {in_file}  >> {changed_in_file}')

    #shellで実行するように各自調整
    cmd = f"python3 {root + contest_name + program} < {changed_in_file}"
    my_process = run(
      cmd,
      shell=True,
      stdout=PIPE, stderr=PIPE
    )

    score = int(my_process.stdout.decode())
    #029では2の累乗ごとに上がって行く為
    return math.log2(score)


def calc_scores(wait_turn: int,can_per_boarder: float,cores: int):
    # n-jobsはCPU並列数
    # それぞれのCPU数にあわせると良い
    scores = joblib.Parallel(n_jobs=cores)(
        joblib.delayed(calc_score_each)(i, wait_turn,can_per_boarder) for i in range(n_files)
    )
    return scores


def objective(trial: optuna.trial.Trial):
    start = time.time()
    cores = multiprocessing.cpu_count()
    # ここに探索したいパラメータを置いておく
    wait_turn = trial.suggest_int("wait_turn", 50, 300)
    can_per_boarder = trial.suggest_float("can_per_boarder", 0.5, 4.0)
    # scoreで計算できるようにする
    scores = calc_scores(wait_turn,can_per_boarder,cores)
    print(f"elapsed: {time.time() - start}")
    return statistics.mean(scores)


if __name__ == "__main__":

    #directionは答えを最大or最小にするかどうか
    study = optuna.create_study(
        direction = "maximize",
        storage=f"sqlite:///{contest_name[:-1]}.db",
        study_name = "tune-param",
        load_if_exists = True,
    )

    #n_trialsは試行回数
    study.optimize(objective, n_trials = 100)



In [ ]:
study = optuna.load_study(storage=f"sqlite:///{contest_name}.db", study_name="tune-param")
optuna.visualization.plot_optimization_history(study)